In [1]:
all_hotels = []

In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

city_dest_id = {}
city_dest_id['London'] = '-2601889'
# city_dest_id['Liverpool'] = '-2601422'
# city_dest_id['Nottingham'] = '-2604469'
city_dest_id['Paris'] = '-1456928'
city_dest_id['Berlin'] = '-1746443'
city_dest_id['Madrid'] = '-390625'
# city_dest_id['Amesterdam'] = '-2140479'
# city_dest_id['Zurich'] = '-2554935'
# city_dest_id['Rome'] = '-126693'
def get_hotels_objects(adults_number, time_interval, city_name):
    
    def get_hotels_from_page(soup):
        hotels = soup.findAll('div', {'data-testid': 'property-card'})
        hotel_list = []
        for hotel in hotels:
            
            hotel_list.append(hotel)
        return hotel_list

    check_in_date = datetime.now().strftime('%Y-%m-%d')
    check_out_date = (datetime.now() + timedelta(days=time_interval)).strftime('%Y-%m-%d')
    
    # Initialize parameters
    url = 'https://www.booking.com/searchresults.en-gb.html'
    params = {
        'ss': city_name,
        'ssne': city_name,
        'ssne_untouched': city_name,
        'efdco': '1',
        'label': 'gog235jc-1FCAEoggI46AdICVgDaFCIAQGYAQm4ARfIAQzYAQHoAQH4AQKIAgGoAgO4ArDuuaEGwAIB0gIkZmJhYjE4YzAtNDdhMy00MmY1LTk2NWItN2UzOTgyNTk1OWEx2AIF4AIB',
        'aid': '397594',
        'lang': 'en-gb',
        'sb': '1',
        'src_elem': 'sb',
        'src': 'searchresults',
        'dest_id': city_dest_id[city_name],
        'dest_type': 'city',
        'checkin': check_in_date,
        'checkout': check_out_date,
        'group_adults': str(adults_number),
        'no_rooms': '1',
        'group_children': '0',
        'offset': 0  # Start with the first page
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 8172.45.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.64 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.5'
    }
    
    # List to store all hotels
    all_hotels = []
    
    # Pagination handling
    while True:
        response = requests.get(url, headers=headers, params=params)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        hotels_on_page = get_hotels_from_page(soup)
        if not hotels_on_page:
            break  # Exit loop if no hotels are found on the current page
    
        all_hotels.extend(hotels_on_page)
        params['offset'] += 1  # Move to the next page
        if params['offset']==5:
            break
    return all_hotels


In [3]:
# duplicate_rows.to_csv("nnhe.csv")

In [9]:
import concurrent.futures
from tqdm import tqdm
with concurrent.futures.ThreadPoolExecutor(56) as executor:
    futures = []
    for city_id in tqdm(city_dest_id.keys()):
        for i in tqdm(range(2,4)):
            for j in tqdm(range(1,6)):
                futures.append(executor.submit(get_hotels_objects, i, j, city_id))
    for future in tqdm(concurrent.futures.as_completed(futures)):
        new_hotels = future.result()
        all_hotels.extend(new_hotels)

100%|██████████| 5/5 [00:00<00:00, 405.91it/s]

100%|██████████| 2/2 [00:00<00:00, 62.61it/s]

100%|██████████| 5/5 [00:00<00:00, 621.69it/s]

100%|██████████| 2/2 [00:00<00:00, 76.48it/s]

100%|██████████| 5/5 [00:00<00:00, 544.09it/s]

100%|██████████| 2/2 [00:00<00:00, 69.36it/s]

100%|██████████| 5/5 [00:00<00:00, 457.86it/s]

100%|██████████| 4/4 [00:00<00:00, 30.61it/s]
40it [03:11,  4.78s/it]


In [10]:
print(len(all_hotels))

11500


In [ ]:
duplicate_rows.to_csv("nnhe.csv")

In [11]:
def extract_feature(hotel, tag, key_dict, value_dict):
    element = hotel.find(tag, {key_dict: value_dict})
    try:
        element_text = element.text.strip()
    except:
        element_text = None
    return element_text

In [12]:
def extract_number_of_beds(hotel):
    try:
        element = hotel.find('div', {'class': 'fc367255e6'}).find_all('div', {'class': 'abf093bdfe'})[-1]
        element_text = element.text.strip()
    except:
        element_text = None
    return element_text

In [13]:
def extract_size(hotel):
    try:
        size_info_div = hotel.find('div', {'data-testid': 'property-card-unit-configuration'})
        spans = size_info_div.find_all('span', class_='a21c5c4883')
        element_text = spans[-1].text.strip() if spans else None
        if element_text[-1] ==  '²':
            return element_text
        else:
            return None
    except:
        return None

In [14]:
hotels_data=[]

In [15]:
for hotel in all_hotels:
    # Extract the hotel name
    name = extract_feature(hotel, 'div', 'data-testid', 'title')
    location = extract_feature(hotel, 'span', 'data-testid', 'address')
    price = extract_feature(hotel, 'span', 'data-testid', 'price-and-discounted-price')
    rating_sample = extract_feature(hotel, 'div', 'class', 'a3b8729ab1 d86cee9b25')
    if rating_sample:
        rating = rating_sample.split(' ')[-1]
    else:
        rating = None
    quality = extract_feature(hotel, 'div', 'class', 'a3b8729ab1 e6208ee469 cb2cbb3ccb')
    review = extract_feature(hotel, 'div', 'class', 'abf093bdfe f45d8e4c32 d935416c47')
    beds = extract_number_of_beds(hotel)
    size = extract_size(hotel)
    distance = extract_feature(hotel, 'span', 'data-testid', 'distance').split(' ')[0]
    room_type = extract_feature(hotel, 'h4', 'class', 'abf093bdfe e8f7c070a7')
    nights_and_adults = extract_feature(hotel, 'div', 'class', 'abf093bdfe f45d8e4c32').split(",")
    nights, adults = nights_and_adults[0], nights_and_adults[1]
    free_cancelation = extract_feature(hotel, 'div', 'class', 'abf093bdfe d068504c75')
    
    
    # Append hotes_data with info about hotel
    hotels_data.append({
        'name': name,
        'location': location,
        'price': price,
        'rating': rating,
        'quality' : quality,
        'review' : review,
        'bed' : beds,
        'size' : size,
        'distance from centure' : distance,
        'room_type' : room_type,
        'nights' : nights,
        'adults' : adults,
        'free_canelation' : free_cancelation
    })

In [ ]:
duplicate_rows.to_csv("nnhe.csv")

In [16]:
print(len(hotels_data))

11500


In [17]:
import pandas as pd
hotels = pd.DataFrame(hotels_data)
df = hotels
duplicate_rows = df[df.duplicated(keep='first')]
print(len(duplicate_rows))

7322


In [18]:
duplicate_rows.to_csv("nnhe.csv")

In [19]:
duplicate_rows.head(5)


,name,location,price,rating,quality,review,bed,size,distance from centure,room_type,nights,adults,free_canelation
25,WIT Hotel,"9th arr., Paris","IRR 7,951,542",7.1,Good,"3,300 reviews",1 double bed,20m²,1.9,Suite,1 night,1 adult,None
27,UCPA SPORT STATION HOSTEL PARIS,"19th arr., Paris","IRR 1,397,677",8.0,Very good,"5,921 reviews",1 single bed,None,4.6,Single Bed in 6-Bed Dormitory Room,1 night,1 adult,None
28,Timhotel Montmartre,"18th arr., Paris","IRR 7,569,083",8.3,Very good,"2,532 reviews",1 single bed,None,3.4,Comfort Single Room,1 night,1 adult,None
29,Hôtel Galileo Champs Elysées,"8th arr., Paris","IRR 6,447,737",8.6,Fabulous,"1,457 reviews",1 large double bed,None,4.2,Classic Room,1 night,1 adult,None
30,Hôtel Le Daum,"12th arr., Paris","IRR 7,913,496",8.3,Very good,"1,273 reviews",1 double bed,None,4.1,Comfort Double Room,1 night,1 adult,None


In [21]:
df['price'] = df['price'].str.replace(',', '')
df['price2'] = pd.to_numeric(df['price'].str.split().str[1])//58000

In [22]:
df.head()

,name,location,price,rating,quality,review,bed,size,distance from centure,room_type,nights,adults,free_canelation,price2
0,WIT Hotel,"9th arr., Paris",IRR 7951542,7.1,Good,"3,300 reviews",1 double bed,20m²,1.9,Suite,1 night,1 adult,None,137
1,UCPA SPORT STATION HOSTEL PARIS,"19th arr., Paris",IRR 1397677,8.0,Very good,"5,921 reviews",1 single bed,None,4.6,Single Bed in 6-Bed Dormitory Room,1 night,1 adult,None,24
2,Lennon by Avalon Hotel Paris Gare du Nord,"10th arr., Paris",IRR 3914698,7.4,Good,"2,242 reviews",1 single bed,None,2.7,Charm Standard Single Room,1 night,1 adult,None,67
3,Generator Paris,"10th arr., Paris",IRR 1269523,7.3,Good,"11,103 reviews",1 bunk bed,None,2.7,Bed in 10-Bed Mixed Dormitory Room,1 night,1 adult,None,21
4,Hôtel Le Daum,"12th arr., Paris",IRR 7913496,8.3,Very good,"1,273 reviews",1 double bed,None,4.1,Comfort Double Room,1 night,1 adult,None,136
